# CNP_func_validation_age

In [23]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

In [37]:
# Config
toy = False
run_all = False



#AWS cloud
# path = '/home/ubuntu/fmriprep/'
# output_dir = '/output/' 

#For Daniel's computer
input_dir = 'data/CNP_func/'
output_dir = 'data/CNP_T1/' 


n_jobs = 20 #amount of cores
cv=4

parcellation = '444'

description = 'CNP_func_validation_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [38]:
# Load data
loaded = np.load(input_dir+'CNP_func_gridsearch_age_18-08-09-22-05-36CNP_func'+parcellation+'_train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']

In [39]:
print(y_train.shape)
print(X_train.shape)
X_train = np.reshape(X_train, (84, X_train.shape[2]))

(84,)
(84, 1, 98346)


In [40]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

## Validation 

Grid search returned the best parameters:

* ANOVA k selection: 'all'
* C parameter of SVR: 1


In [41]:
X_train.shape

(58, 98346)

In [43]:
from sklearn.pipeline import Pipeline

if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]
    X_validation = X_validation[:8]
    y_validation = y_validation[:8]


# Remove features with too low between-subject variance
# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
anova_svr = Pipeline([
            ('anova', SelectKBest(f_regression, k='all')),
            ('svr', LinearSVR(C=1.))])

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]


anova_svr.fit(X_train, y_train)
predictions = anova_svr.predict(X_validation)

# clf = SVR(kernel='linear', C=0.01)
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_validation)



score = mean_absolute_error(y_validation, predictions)
r2 = r2_score(y_validation, predictions)
# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






In [45]:
print(score)
print(predictions)
print(y_validation)

6.382342594339007
[30.81325851 26.03452791 31.42972712 26.83977765 32.81079145 31.8646006
 30.1627071  35.90164305 23.73649735 32.51839141 30.6036088  23.73541716
 26.81080994 30.56967795 34.7209717  35.98615299 28.65561503 31.89597547
 29.43347504 30.27372227 30.15936703 25.00568139 27.19646462 27.03507136
 29.90209371 26.12426744]
[25 29 40 40 23 26 22 36 29 30 32 23 25 24 35 22 31 48 37 28 41 39 35 23
 32 38]


In [46]:
# print('best score: ',grid.best_score_)
# print('best params: ',grid.best_params_)
# print('\n\nfullresults: \n', grid.cv_results_)
# df1 = pd.DataFrame(grid.cv_results_)
# df1

In [47]:


with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('mean absolute error: '+str(score.round(4))+'\n')
    f.write('r2: '+str(r2.round(4))+'\n')
    f.write(str(predictions)+'\n')
#     f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
#     f.write(str(df1)+'\n')
#     f.write(str(age_pred)+'\n')
#     f.write(str(y_train)+'\n')
#     f.write(str(mean_abs_error)+'\n')
    

In [48]:
r2

-0.27706915864013837

In [ ]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [ ]:
# r2 score